# QnA maker服務操作

In [7]:
import http.client, urllib.parse, json, time, sys

# Represents the various elements used to create HTTP request URIs
# for QnA Maker operations.
# From Publish Page
# Example: YOUR-RESOURCE-NAME.azurewebsites.net
# CAUTION: This is not the exact value of HOST field
# HOST trimmed to work with http library
# host = "https://qamakedemo.cognitiveservices.azure.com/qnamaker/v4.0";

host = 'tfb103qna.azurewebsites.net' #"qamakeDEMO.azurewebsites.net";

# Authorization endpoint key
# From Publish Page
endpoint_key = 'b552acca-c911-4fe7-b694-7647469d0320' #"e71e8e5a-e6f1-461b-9602-af667fa12ecb";

# Management APIs postpend the version to the route
# From Publish Page
# Example: /knowledgebases/ZZZ15f8c-d01b-4698-a2de-85b0dbf3358c/generateAnswer
# CAUTION: This is not the exact value after POST
# Part of HOST is prepended to route to work with http library
route = '/qnamaker/knowledgebases/4851aab2-36d7-4069-aa7b-8cbc8049a599/generateAnswer'

# JSON format for passing question to service
question = "{'question': 'how old are you?','top': 3}";

headers = {
    'Authorization': 'EndpointKey ' + endpoint_key,
    'Content-Type': 'application/json'
  }

try:
    conn = http.client.HTTPSConnection(host,port=443)
    conn.request ("POST", route,  question, headers)
    response = conn.getresponse ()
    answer = response.read ()
    print(json.dumps(json.loads(answer), indent=4))

except :
    print ("Unexpected error:", sys.exc_info()[0])
    print ("Unexpected error:", sys.exc_info()[1])

{
    "answers": [
        {
            "questions": [
                "how old are you?"
            ],
            "answer": "i'm forever 18 years old.",
            "score": 100.0,
            "id": 2,
            "source": "Editorial",
            "isDocumentText": false,
            "metadata": [],
            "context": {
                "isContextOnly": false,
                "prompts": []
            }
        }
    ],
    "activeLearningEnabled": false
}


# 內容仲裁服務操作

In [12]:
import os.path
from pprint import pprint
import time
from io import BytesIO
from random import random
import uuid

from azure.cognitiveservices.vision.contentmoderator import ContentModeratorClient
import azure.cognitiveservices.vision.contentmoderator.models
from msrest.authentication import CognitiveServicesCredentials

CONTENT_MODERATOR_ENDPOINT = "https://tfb103content.cognitiveservices.azure.com/"
subscription_key = "3684b9ffede04c158418d0863521b96f"

client = ContentModeratorClient(
    endpoint=CONTENT_MODERATOR_ENDPOINT, 
    credentials=CognitiveServicesCredentials(subscription_key)
)

TEXT_FOLDER = os.getcwd()

# Screen the input text: check for profanity,
# do autocorrect text, and check for personally identifying
# information (PII)
with open(os.path.join(TEXT_FOLDER, 'content_moderator_text_moderation.txt'), "rb") as text_fd:
    screen = client.text_moderation.screen_text(
        text_content_type="text/plain",
        text_content=text_fd,
        language="eng",
        autocorrect=True,
        pii=True
    )
#     assert isinstance(screen, Screen)
    print(screen.as_dict())

{'original_text': 'Is this a grabage email abcdef@abcd.com, phone: 4255550111, IP: 255.255.255.255, 1234 Main Boulevard, Panapolis WA 96555.\nCrap is the profanity here. Is this information PII? phone 2065550111', 'normalized_text': '   grabage email abcdef@abcd.com, phone: 4255550111, IP: 255.255.255.255, 1234 Main Boulevard, Panapolis WA 96555.\nCrap   profanity .   information PII? phone 2065550111', 'auto_corrected_text': 'Is this a garbage email abcdef@abcd.com, phone: 4255550111, IP: 255.255.255.255, 1234 Main Boulevard, Pentapolis WA 96555.\nCrap is the profanity here. Is this information PII? phone 2065550111', 'status': {'code': 3000, 'description': 'OK'}, 'pii': {'email': [{'detected': 'abcdef@abcd.com', 'sub_type': 'Regular', 'text': 'abcdef@abcd.com', 'index': 24}], 'ssn': [], 'ipa': [{'sub_type': 'IPV4', 'text': '255.255.255.255', 'index': 64}], 'phone': [{'country_code': 'US', 'text': '4255550111', 'index': 48}, {'country_code': 'US', 'text': '2065550111', 'index': 181}],